In [1]:
import os
import numpy as np
import numpy.random
import pandas as pd

In [2]:
# Probability for BTW and Oslo Models
BTW_probability = 1
Oslo_probability = 0.5
# Lengths to be investigated
L = [4, 8, 16, 32, 64, 128, 256]

In [3]:
class Pile:
    def __init__(self, length, probability=0.5):
        self.probability = probability
        self.length = length
        self.lattice = np.zeros(length)
        self.gradient = np.zeros(length, dtype=np.int)
        self.threshold = np.round(np.random.random(length)\
                                  + 1.5\
                                  - self.probability).astype(np.int)
        self.grains_dropped = 0
        self.grains_left = 0
        self.height = 0
        self.avalanche_size = 0
        self.cross_over_state = False
    
    def get_grains_dropped(self):
        return self.grains_dropped
    
    def get_grains_left(self):
        return self.grains_left
    
    def get_size(self):
        return self.grains_dropped - self.grains_left
    
    def get_height(self):
        return self.height
    
    def get_avalanche_size(self):
        return self.avalanche_size
    
    def get_gradient(self):
        return self.gradient
    
    def get_threshold(self):
        return self.threshold
    
    def drive(self, site=0):
        '''Adds 1 grain'''
        self.avalanche_size = 0
        self.gradient[site] += 1
        self.height += 1
        self.grains_dropped += 1
        
    def relax(self, site):
        '''Relaxes site gradients according to Oslo model rules'''
        if site == 0:
            self.height -= 1
            self.gradient[site] -= 2
            self.gradient[site + 1] += 1
        elif site == self.length - 1:
            self.gradient[site] -= 1
            self.gradient[site-1] += 1
            self.cross_over_state = True
            self.grains_left += 1
        else:
            self.gradient[site] -= 2
            self.gradient[site-1] += 1
            self.gradient[site+1] += 1
        self.update_threshold(site)    
    
    def update_threshold(self, site):
        '''Updates threshold to 1 with probability p and 2 otherwise'''
        self.threshold[site] = round(1.5\
                                       - self.probability\
                                       + np.random.random())
    
    def find_unstable_sites(self):
        return np.where(np.greater(self.gradient, self.threshold))[0]
    
    def drop_grain(self, site=0):
        '''Starts 1 drive process and relaxes until stable'''
        self.drive(site)
        while True:
            unstable_sites = self.find_unstable_sites()
            if unstable_sites.size < 1:
                break
            else:
                for i in unstable_sites:
                    self.avalanche_size += 1
                    self.relax(i)


## Task 1: Testing the Program

### Test: $p=1$, BTW Model

Let $p=1$, and check the following:
- All threshold values are 1
- All gradients are 1 in the steady state.

In [4]:
BTW_Model = Pile(8, BTW_probability)
print('Threshold: {}'.format(BTW_Model.get_threshold()))

Threshold: [1 1 1 1 1 1 1 1]


In [5]:
for i in range(100): #100 enough to reach steady state
    BTW_Model.drop_grain()
print('''
    Thresholds: {}
    Gradients: {}
    Height: {}'''
    .format(BTW_Model.get_threshold(), 
            BTW_Model.get_gradient(), 
            BTW_Model.get_height()))


    Thresholds: [1 1 1 1 1 1 1 1]
    Gradients: [1 1 1 1 1 1 1 1]
    Height: 8


Threshold, gradient and height as expected for BTW model.

### Test: $p=0$

Let $p=0$, and check the following:
- All threshold values are 0
- All gradients are 2 in the steady state.

Essentially double the values in BTW Model

In [6]:
Zero_Model = Pile(8, 0)
for i in range(100):
    Zero_Model.drop_grain()
print('''
    Thresholds: {}
    Gradients: {}
    Height: {}'''
    .format(Zero_Model.get_threshold(), 
            Zero_Model.get_gradient(), 
            Zero_Model.get_height()))


    Thresholds: [2 2 2 2 2 2 2 2]
    Gradients: [2 2 2 2 2 2 2 2]
    Height: 16


Threshold, gradient and height as expected.

### Test: Average Height at Steady State

In [7]:
Oslo_Model_16 = Pile(16, Oslo_probability)
Oslo_Model_32 = Pile(32, Oslo_probability)
heights_16 = []
heights_32 = []
for i in range(50000):
    Oslo_Model_16.drop_grain()
    Oslo_Model_32.drop_grain()
    heights_16.append(Oslo_Model_16.get_height())
    heights_32.append(Oslo_Model_32.get_height())
print('''
    Height for L=16: {}
    Height for L=32: {}'''
    .format(np.mean(heights_16), np.mean(heights_32)))


    Height for L=16: 26.48004
    Height for L=32: 53.54844


In the steady state, the Oslo models results in $\langle h_{16} \rangle = 26.5$ and $\langle h_{32} \rangle = 53.9$ as expected.

## Creating Dataset

We aim to collect data until the steady state has been reached. We use cross-over time as an estimate for the time required to reach steady state. We only consider cross-over time for the largest pile L=512 since time for all other pile sizes will be smaller.

In [302]:
Oslo_Model_512 = Pile(512)
while not Oslo_Model_512.cross_over_state:
    Oslo_Model_512.drop_grain()
Oslo_Model_512.get_grains_dropped()

225481

We will simulate 5e5 grains, roughly double the cross-over time measured above. The data will be generated and saved in a npy file.

In [309]:
n = 5e5
piles = [Pile(l) for l in L]
data = []
for i in range(int(n)):
    for pile in piles:
        pile.drop_grain()
    data.append([[pile.get_height(), pile.get_avalanche_size()] for pile in piles])
data_np = np.array(data)
np.save('data1', data_np)

We will read

## Task 2: Height of Pile

### Task 2a: Pile Height as a Function of Time